# 🎉 Smart Recommender – Sentence‑Transformer (SBERT)

In the previous notebook we built a TF‑IDF baseline.  
Now we’ll replace the **bag‑of‑words** representation with **dense semantic embeddings** from a pretrained sentence‑transformer.  
The workflow is exactly the same as before, only the vectorisation step changes:
1️⃣ Install `sentence‑transformers`.  
2️⃣ Load a lightweight model (`all‑MiniLM‑L6‑v2`).  
3️⃣ Encode every movie’s combined text into a 384‑dim vector.  
4️⃣ At query time we encode the user’s sentence and compute cosine similarity against all movie vectors.  
5️⃣ Return the top‑N most similar movies.

Because the model already knows **semantic relationships** (e.g., *dream* ≈ *sleep*, *heist* ≈ *robbery*), the recommendations become **much more relevant** than the TF‑IDF baseline.

In [ ]:
# 1️⃣ Install the library (run once)
!pip install -q sentence-transformers

import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

print("✅ sentence‑transformers installed and imports ready!")

## 📂 Load the **cleaned** TMDb data

We reuse the `movies_clean` DataFrame you created in the TF‑IDF notebook.  
If you are running this notebook **after** the previous one in the same session, the variable already exists.  
Otherwise, just re‑run the loading cells from `01_tmdb_exploration.ipynb` (they are tiny).

In [ ]:
# Load the CSV we saved at the end of the TF‑IDF notebook
movies_clean = pd.read_csv('tmdb_clean.csv')

# Re‑create the combined text column (just in case we started a fresh kernel)
def create_combined_text(row):
    parts = [str(row['overview'])]
    if row['genres_list']:
        parts.append(' '.join(row['genres_list']))
    if row['keywords_list']:
        parts.append(' '.join(row['keywords_list']))
    return ' '.join(parts)

movies_clean['combined_text'] = movies_clean.apply(create_combined_text, axis=1)

print(f"✅ Loaded {len(movies_clean)} movies with combined text ready.")

## 🤖 2️⃣ Load a pretrained sentence‑transformer

We pick **`all‑MiniLM‑L6‑v2`** – 384‑dim, ~30 MB, works well on CPUs and GPUs.  
If you have a GPU you’ll see a speed boost, but the model runs fine on a regular Colab CPU runtime.

In [ ]:
# Load the model (download ~30 MB on first run)
model = SentenceTransformer('all-MiniLM-L6-v2')

print("✅ Model loaded! Vocabulary size:", len(model.get_vocab()))

## 📦 3️⃣ Encode **all** movies once (this is the heavy step)

We turn every `combined_text` into a 384‑dim dense vector and store it in a NumPy array.  
The resulting matrix has shape `(num_movies, 384)` – tiny compared to the original TF‑IDF matrix.

In [ ]:
# Encode all movies – this may take ~30‑45 seconds for ~5k rows
movie_embeddings = model.encode(
    movies_clean['combined_text'].tolist(),
    batch_size=64,
    show_progress_bar=True,
    convert_to_numpy=True
)

print(f"✅ Encoded {movie_embeddings.shape[0]} movies → {movie_embeddings.shape[1]}‑dim vectors")

## 🔎 4️⃣ Recommendation function (query → top‑N movies)

The function does three things:
1️⃣ Encode the user query with the same model.
2️⃣ Compute **cosine similarity** against the pre‑computed movie embeddings.
3️⃣ Return the `top_n` titles with their similarity scores (0‑1).

Because the embeddings are dense, cosine similarity is a **single matrix multiplication** – extremely fast.

In [ ]:
def recommend_by_embedding(query: str, top_n: int = 5):
    """Return the most similar movies for a free‑text query.
    
    Parameters
    ----------
    query : str
        User’s natural‑language request (e.g., "mind‑bending sci‑fi with dreams").
    top_n : int, optional
        Number of movies to return (default 5).
    """
    # 1️⃣ Encode the query
    q_vec = model.encode([query], convert_to_numpy=True)

    # 2️⃣ Cosine similarity against all movie vectors
    sims = cosine_similarity(q_vec, movie_embeddings).flatten()

    # 3️⃣ Get top‑N indices (largest similarity first)
    top_idx = sims.argsort()[::-1][:top_n]

    # 4️⃣ Build a friendly output list
    results = []
    for i in top_idx:
        title = movies_clean.iloc[i]['title']
        score = round(sims[i], 4)
        results.append((title, score))
    return results

# Quick sanity check
print("🔎 Test query → results:")
for t, s in recommend_by_embedding('mind‑bending sci‑fi with dreams'):
    print(f"  • {t} (score={s})")

## 📊 5️⃣ Quick evaluation – try a few different queries

Run the cell below and replace the strings with any natural‑language request you like.  
You should see **semantically relevant** movies (e.g., *Inception*, *Paprika*, *The Matrix*, *Interstellar*, etc.).

In [ ]:
queries = [
    "mind‑bending sci‑fi with dreams",
    "family friendly animated adventure",
    "dark thriller about a detective",
    "movie about time travel and paradoxes",
    "light romantic comedy set in Paris"
]

for q in queries:
    print(f"\n🔎 Query: '{q}'")
    for title, score in recommend_by_embedding(q, top_n=5):
        print(f"  • {title} (score={score})")

## 💾 6️⃣ Persist the embeddings (optional but handy for production)

If you plan to serve the recommender from a web service, you don’t want to re‑encode all movies on every start‑up.  
Save the dense matrix and the title list to disk – they can be loaded instantly later.

```python
# Save
np.save('movie_embeddings.npy', movie_embeddings)
movies_clean[['title']].to_csv('movie_titles.csv', index=False)

# Load (in a new session)
movie_embeddings = np.load('movie_embeddings.npy')
titles = pd.read_csv('movie_titles.csv')['title'].tolist()
```

## 🚀 7️⃣ Next step – expose the recommender as a tiny API (FastAPI)

Below is a **minimal FastAPI** server you can copy into `services/recommender-service/main.py`.  
It loads the saved embeddings and answers `POST /recommend` with JSON.

```python
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

app = FastAPI()

class RecRequest(BaseModel):
    query: str
    top_n: int = 5

# Load model & data once at startup
model = SentenceTransformer('all-MiniLM-L6-v2')
movie_embeddings = np.load('movie_embeddings.npy')
titles = pd.read_csv('movie_titles.csv')['title'].tolist()

@app.post('/recommend')
def recommend(req: RecRequest):
    q_vec = model.encode([req.query], convert_to_numpy=True)
    sims = cosine_similarity(q_vec, movie_embeddings).flatten()
    top_idx = sims.argsort()[::-1][:req.top_n]
    results = [{"title": titles[i], "score": round(float(sims[i]), 4)} for i in top_idx]
    return {"query": req.query, "results": results}
```

Run the service with Docker (or `uvicorn main:app --host 0.0.0.0 --port 8000`).  
Your frontend can now call `POST /recommend` and get instant, semantically‑aware suggestions.

---
### 🎉 You’re done!

You now have:
1. **A TF‑IDF baseline** (already built).
2. **A smart SBERT‑based recommender** that understands meaning.
3. **Persistence** of embeddings for fast start‑up.
4. **A minimal FastAPI wrapper** ready to be containerised.

Feel free to experiment:
- Try a larger model (`all‑distilroberta‑v1`) for even richer semantics.
- Fine‑tune the sentence‑transformer on your own movie‑specific corpus (optional).
- Add genre/keyword weighting in the final similarity (e.g., `0.6*embed + 0.4*tfidf`).

Enjoy building the **Movie Discovery Assistant**! 🚀